## HW Requirement

• Implement the code for the 2-layer neural networks in CS231n 
2021 version with PyTorch (or TensorFlow). 

• Once you have the code (regardless of which framework you 
choose above), you will apply your own data.  The training and test 
dataset is 80%:20%.

• You need to run the code with the following hyperparameter 
settings:

✓ Activation function: tanh, ReLU

✓ Data preprocessing

✓ Initial weights: small random number, Xavier or Kaiming/MSRA 
Initialization

✓ Loss function: without or with the regularization term 
(L2), λ = 
0.001 or 0.0001
$$ E(w) = \frac{1}{N}\sum^{N}_{c=1}[𝑓(X^c, w) −y^c]^2 
 + \lambda[\sum^{p}_{i=0}(w^{o}_{i})^2
 + \sum_{i=1}^{p}\sum_{j=0}^{m}(w_{ij}^H)^2]
$$
✓ Optimizer: gradient descent, Momentum, Adam

✓ Learning epochs: 100, 200, 300

✓ Amount of hidden nodes: 5, 8, 11

✓ Learning rate decay schedule: none and cosine

✓ Ensembles: top 3 models

## Model

In [1]:
import torch
from torch import nn, optim, Generator
from torch.utils.data import DataLoader, Dataset, random_split
from sys import stdout


c:\VS_Workplace\NN\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from typing import Iterable, Callable, Type
from operator import mul

def product(nums: Iterable[Type], func: Callable[[Type, Type], Type] = mul):
    def _product(nums):
        nonlocal func
        if len(nums) == 1:
            return nums[0]
        return func(nums[-1], _product(nums[:-1]))
    try:
        return _product(nums)
    except Exception as e:
        raise e

In [3]:
ACTIVES = {
    "relu": nn.ReLU,
    "tanh": nn.Tanh
}
INIT_FUNCS = {
    "small_random": lambda x: nn.init.normal_(tensor=x, mean=0, std=0.01),
    "xavier": lambda x: nn.init.xavier_uniform_(tensor=x) if len(x.shape) > 1 else None,
    "kaiming": lambda x: nn.init.kaiming_uniform_(tensor=x, nonlinearity='relu') if len(x.shape) > 1 else None
}
OPTIM_FUNCS = {
    "sgd": optim.SGD,
    "momentum": lambda param, lr, weight_decay: optim.SGD(params=param, lr=lr, momentum=0.9, weight_decay=weight_decay),
    "adam": optim.Adam
}
SCHEDULERS = {
    "cos": lambda opt: torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=opt, T_max=200)
}


In [5]:
from collections.abc import Callable
class TwoLayerNetwork(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_classes: int, init_method:Callable, active_func:nn.modules.module.Module) -> None:
        super(TwoLayerNetwork, self).__init__()
        self.input_size = input_size
        self.hidden_size= hidden_size
        ## first layer
        self.fc1 = nn.Linear(input_size, hidden_size)
        ## activation
        self.active_func = active_func()
        ## initialize
        for param in self.parameters():
            init_method(param)
        ## second layer
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.active_func(out)
        out = self.fc2(out)
        return out


In [6]:
def train(model: TwoLayerNetwork, opt: nn.Module, device: str, epochs: int, learning_rate: float, trainloader: DataLoader, valloader: DataLoader, criterion: nn.modules.loss._Loss, sched: optim.lr_scheduler._LRScheduler, weight_decay:float):
    model.to(device)
    optimizer = opt(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = sched(optimizer) if sched else None
    if epochs < 1:
        raise ValueError("Invalid epoch!!")
    else:
        epochs = int(epochs)
    # Train the model
    for epoch in range(epochs):
        train_loss = 0.0
        train_correct = 0
        model.train()
        for X, y in trainloader:
            X = X.view(-1, model.input_size).to(device)
            y = y.to(device)
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_correct += (predicted == y).sum().item()
        train_loss /= len(trainloader.dataset)
        train_accuracy = 100. * train_correct / len(trainloader.dataset)

        # Validate the model
        val_loss = 0.0
        val_correct = 0
        model.eval()
        with torch.no_grad():
            for X, y in valloader:
                X = X.view(-1, model.input_size).to(device)
                y = y.to(device)
                outputs = model(X)
                loss = criterion(outputs, y)
                val_loss += loss.item() * X.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_correct += (predicted == y).sum().item()
            val_loss /= len(valloader.dataset)
            val_accuracy = 100. * val_correct / len(valloader.dataset)
        if scheduler:
            scheduler.step()
        # Print epoch statistics
        stdout.write('Epoch [{}/{}], Train Loss: {:.4f}, Train Accuracy: {:.2f}%, Val Loss: {:.4f}, Val Accuracy: {:.2f}%\n'
              .format(epoch+1, epochs, train_loss, train_accuracy, val_loss, val_accuracy))


In [14]:
def test(model:nn.Module, device:str, testloader:DataLoader):
    val_correct = 0
    model.eval()
    with torch.no_grad():
        for X, y in testloader:
            X = X.view(-1, model.input_size).to(device)
            y = y.to(device)
            outputs = model(X)
            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == y).sum().item()
        val_accuracy = 100. * val_correct / len(testloader.dataset)
        stdout.write('Val Accuracy: {:.2f}%\n'.format(val_accuracy))

# Dataset

### pytorch dataset

In [10]:
# load pytorch dataset

from torchvision import datasets, transforms


def getPytorchData():
    # preprocess: flatten, normalize, drop 90%, split
    transform = transforms.transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    trainset = datasets.FashionMNIST(
        root="./data/", train=True, download=False, transform=transform)
    datum_size = product(trainset[0][0].size())
    class_amount = len(trainset.classes)
    testset = datasets.FashionMNIST(
        root="./data/", train=False, download=False, transform=transform)
    # Split the training set into training and validation sets
    train_count = int(0.08 * len(trainset))
    valid_count = int(0.02 * len(trainset))
    print(train_count, valid_count, len(testset))
    trainset, valset, _ = random_split(
        trainset, (train_count, valid_count, len(trainset)-train_count-valid_count), Generator().manual_seed(42))
    # Create data loaders to load the data in batches
    trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
    valloader = DataLoader(valset, batch_size=32, shuffle=True)
    testloader = DataLoader(testset, batch_size=32, shuffle=True)
    return trainloader, valloader, testloader, datum_size, class_amount


### customized pytorch dataset

In [13]:
import pandas as pd
import numpy as np
class HotelReservationDataset(Dataset):
    """Hotel Reservation dataset."""

    def __init__(self, csv_path):
        """
        Args:
            csv_path (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        # 19
        reservations = pd.read_csv(csv_path)
        # 5
        for col in map(lambda x: x[0], filter(lambda x:x[1]=="O", reservations.dtypes.items())):
            d = dict((j, i) for i, j in enumerate(reservations[col].value_counts().index))
            setattr(self, f"labels_of_{col}", d.keys())
            reservations[col]=reservations[col].map(d.__getitem__)
        # 17(drop id)
        self.feature = torch.from_numpy(reservations.iloc[:, 1:-1].to_numpy(dtype=np.float32))
        # two status
        self.booking_status = torch.reshape(torch.tensor(reservations.iloc[:, -1:].to_numpy()), shape=(len(self.feature),))
        self.classes = list(getattr(self, f"labels_of_{reservations.columns[-1]}"))
    def __len__(self):
        return len(self.booking_status)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return self.feature[idx], self.booking_status[idx]

# kaggle: ahsan81/hotel-reservations-classification-dataset
def getCustomizedData():
    # preprocess
    dataset = HotelReservationDataset(
        csv_path=r"D:\dataset\archive\Hotel Reservations.csv")
    class_amount = len(dataset.classes)
    # train test split
    train_count = int(0.7 * len(dataset))
    valid_count = int(0.2 * len(dataset))
    test_count = len(dataset) - train_count - valid_count
    print(train_count, valid_count, test_count)
    trainset, valset, testset = random_split(
        dataset, (train_count, valid_count, test_count), Generator().manual_seed(42))
    datum_size = product(trainset[0][0].size())
    # set loaders
    trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
    valloader = DataLoader(valset, batch_size=32, shuffle=True)
    testloader = DataLoader(testset, batch_size=32, shuffle=True)
    return trainloader, valloader, testloader, datum_size, class_amount


### kaggle dataset

In [3]:
# download data(zipped csv) from kaggle with username and apikey
import os
import json
with open("kaggle.json", "r") as j:
    for (k, v) in json.load(j).items():
        os.environ[k] = v
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
# https://www.kaggle.com/datasets/uciml/iris/download?datasetVersionNumber=2
# owner/datasetname
api.dataset_download_files('uciml/iris', path="./data/")


## Training

In [7]:
def training_schedule():
    stdout = open("./result/hw1.txt", "w")
    # processor
    device = "cuda" if torch.cuda.is_available(
    ) else "mps" if torch.backends.mps.is_available() else "cpu"
    # hyper parameters
    trainloader, valloader, testloader, input_size, output_size = getPytorchData()
    learning_rate = 0.001
    criterion = nn.CrossEntropyLoss()
    # ✓ Amount of hidden nodes: 5, 8, 11
    for hidden_size in (5, 8, 11):
        # ✓ Learning epochs: 100, 200, 300
        for epochs in (100, 200, 300):
            # Create model, optimizer, scheduler
            for (init, method) in INIT_FUNCS.items():
                for (active, func) in ACTIVES.items():
                    # ✓ Activation function: tanh, ReLU
                    # ✓ Initial weights: small random number, Xavier or Kaiming/MSRA Initialization
                    model = TwoLayerNetwork(input_size, hidden_size, output_size,
                                            init_method=method, active_func=func).to(device)
                    # ✓ Optimizer: gradient descent, Momentum, Adam
                    for (optimize, optm) in OPTIM_FUNCS.items():
                        # ✓ Learning rate decay schedule: none and cosine
                        for (schedule, schd) in SCHEDULERS.items():
                            # ✓ Loss function: without or with L2, λ = 0.001 or 0.0001
                            for weight_decay in (0.0, 0.001, 0.0001):
                                print(hidden_size, epochs, init, active,
                                      optimize, schedule, weight_decay,  "start")
                                train(model=model, opt=optm, device=device, epochs=epochs, learning_rate=learning_rate,
                                      trainloader=trainloader, valloader=valloader, criterion=criterion, sched=schd, weight_decay=weight_decay)
                                test(model=model, device=device,
                                     testloader=testloader)


5 100 small_random relu sgd cos
5 100 small_random relu momentum cos
5 100 small_random relu adam cos
5 100 small_random tanh sgd cos
5 100 small_random tanh momentum cos
5 100 small_random tanh adam cos
5 100 xavier relu sgd cos
5 100 xavier relu momentum cos
5 100 xavier relu adam cos
5 100 xavier tanh sgd cos
5 100 xavier tanh momentum cos
5 100 xavier tanh adam cos
5 100 kaiming relu sgd cos
5 100 kaiming relu momentum cos
5 100 kaiming relu adam cos
5 100 kaiming tanh sgd cos
5 100 kaiming tanh momentum cos
5 100 kaiming tanh adam cos
5 200 small_random relu sgd cos
5 200 small_random relu momentum cos
5 200 small_random relu adam cos
5 200 small_random tanh sgd cos
5 200 small_random tanh momentum cos
5 200 small_random tanh adam cos
5 200 xavier relu sgd cos
5 200 xavier relu momentum cos
5 200 xavier relu adam cos
5 200 xavier tanh sgd cos
5 200 xavier tanh momentum cos
5 200 xavier tanh adam cos
5 200 kaiming relu sgd cos
5 200 kaiming relu momentum cos
5 200 kaiming relu adam

### setting

In [11]:
# processor
device = "cuda" if torch.cuda.is_available(
) else "mps" if torch.backends.mps.is_available() else "cpu"
# hyper parameters
trainloader, valloader, testloader, input_size, output_size = getPytorchData()
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()

hidden_size = 5
epochs = 100
init = "small_random"
method = INIT_FUNCS[init]
active = "relu"
func = ACTIVES[active]
optimize = "sgd"
optm = OPTIM_FUNCS[optimize]
schedule = None
schd = schedule
weight_decay = 0.0



4800 1200 10000


### train

In [15]:
counter = 0

In [16]:
for _hidden_size in (5, 8, 11):
    counter += 1
    model = TwoLayerNetwork(input_size=input_size, hidden_size=_hidden_size,
                            num_classes=output_size, init_method=method, active_func=func)
    with open(f"./data/{counter}.txt", "w+") as f:
        stdout = f
        stdout.write(f"{counter}: {_hidden_size}, {epochs}, {init}, {active}, {optimize}, {schedule}, {weight_decay}\n")
        train(model=model, opt=optm, device=device, epochs=epochs, learning_rate=learning_rate,
              trainloader=trainloader, valloader=valloader, criterion=criterion, sched=schd, weight_decay=weight_decay)
        test(model=model, device=device, testloader=testloader)


In [17]:
# ✓ Learning epochs: 100, 200, 300
for _epochs in (100, 200, 300):
    counter += 1
    model = TwoLayerNetwork(input_size=input_size, hidden_size=hidden_size,
                            num_classes=output_size, init_method=method, active_func=func)
    with open(f"./data/{counter}.txt", "w+") as f:
        stdout = f
        stdout.write(
            f"{counter}: {hidden_size}, {_epochs}, {init}, {active}, {optimize}, {schedule}, {weight_decay}\n")
        train(model=model, opt=optm, device=device, epochs=_epochs, learning_rate=learning_rate,
              trainloader=trainloader, valloader=valloader, criterion=criterion, sched=schd, weight_decay=weight_decay)
        test(model=model, device=device, testloader=testloader)


In [18]:
# ✓ Initial weights: small random number, Xavier or Kaiming/MSRA Initialization
for (_init, _method) in INIT_FUNCS.items():
    counter += 1
    model = TwoLayerNetwork(input_size=input_size, hidden_size=hidden_size,
                            num_classes=output_size, init_method=_method, active_func=func)
    with open(f"./data/{counter}.txt", "w+") as f:
        stdout = f
        stdout.write(
            f"{counter}: {hidden_size}, {epochs}, {_init}, {active}, {optimize}, {schedule}, {weight_decay}\n")
        train(model=model, opt=optm, device=device, epochs=epochs, learning_rate=learning_rate,
              trainloader=trainloader, valloader=valloader, criterion=criterion, sched=schd, weight_decay=weight_decay)
        test(model=model, device=device, testloader=testloader)


In [19]:
# ✓ Activation function: tanh, ReLU
for (_active, _func) in ACTIVES.items():
    counter += 1
    model = TwoLayerNetwork(input_size=input_size, hidden_size=hidden_size,
                            num_classes=output_size, init_method=method, active_func=_func)
    with open(f"./data/{counter}.txt", "w+") as f:
        stdout = f
        stdout.write(f"{counter}: {hidden_size}, {epochs}, {init}, {_active}, {optimize}, {schedule}, {weight_decay}\n")
        train(model=model, opt=optm, device=device, epochs=epochs, learning_rate=learning_rate,
              trainloader=trainloader, valloader=valloader, criterion=criterion, sched=schd, weight_decay=weight_decay)
        test(model=model, device=device, testloader=testloader)

In [20]:
# ✓ Optimizer: gradient descent, Momentum, Adam
for (_optimize, _optm) in OPTIM_FUNCS.items():
    counter += 1
    model = TwoLayerNetwork(input_size=input_size, hidden_size=hidden_size,
                            num_classes=output_size, init_method=method, active_func=func)
    with open(f"./data/{counter}.txt", "w+") as f:
        stdout = f
        stdout.write(f"{counter}: {hidden_size}, {epochs}, {init}, {active}, {_optimize}, {schedule}, {weight_decay}\n")
        train(model=model, opt=_optm, device=device, epochs=epochs, learning_rate=learning_rate,
              trainloader=trainloader, valloader=valloader, criterion=criterion, sched=schd, weight_decay=weight_decay)
        test(model=model, device=device, testloader=testloader)

In [21]:
# ✓ Learning rate decay schedule: none and cosine
for (_schedule, _schd) in SCHEDULERS.items():
    counter += 1
    model = TwoLayerNetwork(input_size=input_size, hidden_size=hidden_size,
                            num_classes=output_size, init_method=method, active_func=func)
    with open(f"./data/{counter}.txt", "w+") as f:
        stdout = f
        stdout.write(f"{counter}: {hidden_size}, {epochs}, {init}, {active}, {optimize}, {_schedule}, {weight_decay}\n")
        train(model=model, opt=optm, device=device, epochs=epochs, learning_rate=learning_rate,
              trainloader=trainloader, valloader=valloader, criterion=criterion, sched=_schd, weight_decay=weight_decay)
        test(model=model, device=device, testloader=testloader)


In [22]:
# ✓ Loss function: without or with L2, λ = 0.001 or 0.0001
for _weight_decay in (0.0, 0.001, 0.0001):
    counter += 1
    model = TwoLayerNetwork(input_size=input_size, hidden_size=hidden_size,
                            num_classes=output_size, init_method=method, active_func=func)
    with open(f"./data/{counter}.txt", "w+") as f:
        stdout = f
        stdout.write(f"{counter}: {hidden_size}, {epochs}, {init}, {active}, {optimize}, {schedule}, {_weight_decay}\n")
        train(model=model, opt=optm, device=device, epochs=epochs, learning_rate=learning_rate,
              trainloader=trainloader, valloader=valloader, criterion=criterion, sched=schd, weight_decay=_weight_decay)
        test(model=model, device=device, testloader=testloader)


In [5]:
import re

In [1]:
import pandas as pd

In [ ]:
for i in range(1, 19):
    with open(f"./data/{counter}.txt") as f:
        pd.

In [11]:

with open(f"./data/1.txt", "r") as f:
    l = list(f.readlines())
    l[]
    accuracy = l[len(l) - 1][14:-2]


In [12]:
len("Val Accuracy: ")

14